USE "py -m pip install"

In [ ]:
import json
import pandas as pd

In [ ]:
# Load the GeoJSON file
with open(r'\\Nt05\00\0005\CMU_Docs\_Dallas Design Sprint\research\DataSets\BuildingData\w100_n35_w095_n30.geojson', 'r') as f:
    geojson_data = json.load(f)

# Extract features
features = geojson_data['features']
print(f"Total features: {len(features)}")
print(f"\nGeoJSON structure keys: {geojson_data.keys()}")


Total features: 6801377

GeoJSON structure keys: dict_keys(['type', 'name', 'crs', 'features'])


INSPECT GEOJSON FILE

In [15]:
first_ten_features = geojson_data['features'][:10]

# Create a new GeoJSON object for clean printing (optional, but good practice)
preview = {
    "type": "FeatureCollection",
    "features": first_ten_features
}

#Print the result as formatted JSON to the console
print(json.dumps(preview, indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "source": "osm",
        "id": "862371286",
        "height": 13.541568756103516,
        "var": 3.9736480712890625,
        "region": "USA"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -10813498.146723678,
              3592753.929109247
            ],
            [
              -10813448.854453154,
              3592749.16527079
            ],
            [
              -10813447.12900105,
              3592766.9390577604
            ],
            [
              -10813496.42127157,
              3592771.7029029937
            ],
            [
              -10813498.146723678,
              3592753.929109247
            ]
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "source": "osm",
        "id": "862371288",
        "height": 6.9355316162109375

In [6]:
# Extract properties from features into a DataFrame
properties_list = [feature['properties'] for feature in features]
df = pd.DataFrame(properties_list)

# Display columns
print("Columns in the dataset:")
print(df.columns.tolist())
print(f"\nDataFrame shape: {df.shape}")


Columns in the dataset:
['source', 'id', 'height', 'var', 'region']

DataFrame shape: (6801377, 5)


In [7]:
# Display first 10 rows
print("\nFirst 10 rows:")
print(df.head(10).to_string())



First 10 rows:
  source         id     height           var region
0    osm  862371286  13.541569  3.973648e+00    USA
1    osm  862371288   6.935532  1.165594e+00    USA
2    osm  862371291   1.416382  1.248773e+00    USA
3    osm  862371292   1.816897  5.625646e-01    USA
4    osm  862371293   0.009286  4.759677e-06    USA
5    osm  862371294   1.740389  6.875433e-01    USA
6    osm  985470816   4.730319  8.153419e-01    USA
7    osm  985470817   0.009177  2.781089e-07    USA
8    osm  985470818   0.009225  1.604349e-07    USA
9    osm  990259885   1.343309  2.975430e-01    USA


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import folium
import plotly.express as px

In [11]:
# Add geometries to the DataFrame (from the original features list)
geometries = [feature.get('geometry', {}) for feature in features]
df['geometry'] = geometries

# Helper: compute a simple centroid for many geometry types
def get_centroid_coords(geom):
    if not geom:
        return (None, None)
    t = geom.get('type')
    coords = geom.get('coordinates')
    if coords is None:
        return (None, None)
    try:
        if t == 'Point':
            lon, lat = coords[0], coords[1]
            return lon, lat
        if t == 'LineString' or t == 'Polygon':
            pts = coords if t == 'LineString' else coords[0]
            xs = [p[0] for p in pts]
            ys = [p[1] for p in pts]
            return sum(xs)/len(xs), sum(ys)/len(ys)
        if t == 'MultiPolygon':
            # use first polygon's first ring
            pts = coords[0][0]
            xs = [p[0] for p in pts]
            ys = [p[1] for p in pts]
            return sum(xs)/len(xs), sum(ys)/len(ys)
    except Exception:
        return (None, None)
    return (None, None)


In [12]:
# Compute centroids and attach columns
# Use a safe iteration in case some geometries are missing
lon_lat = [get_centroid_coords(g) for g in df['geometry']]
# Ensure same length
if len(lon_lat) == len(df):
    df['lon'], df['lat'] = zip(*lon_lat)
else:
    df['lon'] = [c[0] for c in lon_lat]
    df['lat'] = [c[1] for c in lon_lat]

print('\nFirst 10 rows with computed lon/lat:')
print(df[['lon','lat']].head(10).to_string())


First 10 rows with computed lon/lat:
            lon           lat
0 -1.081348e+07  3.592759e+06
1 -1.081369e+07  3.592368e+06
2 -1.081381e+07  3.592107e+06
3 -1.081380e+07  3.592134e+06
4 -1.081378e+07  3.592214e+06
5 -1.081354e+07  3.592235e+06
6 -1.081915e+07  3.589854e+06
7 -1.081914e+07  3.589816e+06
8 -1.081916e+07  3.589819e+06
9 -1.080651e+07  3.596958e+06


In [13]:
# Plot: try plotly (interactive), then folium, then matplotlib fallback
subset = df.head(10).dropna(subset=['lat','lon'])
if subset.empty:
    print('No coordinates available for the first 10 features to plot.')
else:
    try:
        import plotly.express as px
        hover_col = None
        # choose a reasonable hover column if present
        for c in ('name','id','title'):
            if c in subset.columns:
                hover_col = c; break
        # if no hover_col, pass None
        fig = px.scatter_geo(subset, lon='lon', lat='lat', hover_name=hover_col, scope='world')
        fig.update_layout(title='First 10 GeoJSON Features (centroids)')
        fig.show()
    except Exception as e1:
        try:
            import folium
            m = folium.Map(location=[subset['lat'].mean(), subset['lon'].mean()], zoom_start=13)
            for i, r in subset.iterrows():
                popup = ''
                # try to show a useful property if available
                for p in ('name','id','title'):
                    if p in r.index:
                        popup = str(r[p]); break
                folium.Marker([r['lat'], r['lon']], popup=popup or str(i)).add_to(m)
            display(m)
        except Exception as e2:
            import matplotlib.pyplot as plt
            plt.figure(figsize=(8,6))
            plt.scatter(subset['lon'], subset['lat'])
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')
            plt.title('First 10 GeoJSON Features (centroids)')
            for i, r in subset.iterrows():
                plt.text(r['lon'], r['lat'], str(i), fontsize=9)
            plt.grid(True)
            plt.show()
            print('Plotted with matplotlib fallback. Install plotly or folium for interactive maps.')


In [14]:
# Summary of geometry types and whether Polygon/MultiPolygon include Z (no coordinates printed)
try:
    features  # use features loaded earlier in the notebook
except NameError:
    # fallback: try to load from the same path used above
    import json
    path = r'\\Nt05\\00\\0005\\CMU_Docs\\_Dallas Design Sprint\\research\\DataSets\\BuildingData\\w100_n35_w095_n30.geojson'
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    features = data.get('features', [])

from collections import defaultdict

def coords_have_z(coords):
    if coords is None:
        return False
    if isinstance(coords, (list, tuple)):
        if len(coords) == 0:
            return False
        if isinstance(coords[0], (int, float)):
            return len(coords) >= 3
        return any(coords_have_z(c) for c in coords)
    return False

type_counts = defaultdict(int)
polygon_z_counts = defaultdict(lambda: {'with_z':0, 'without_z':0})

for feat in features:
    geom = feat.get('geometry') if feat else None
    gtype = geom.get('type') if geom else 'None'
    type_counts[gtype] += 1
    if gtype in ('Polygon', 'MultiPolygon'):
        has_z = coords_have_z(geom.get('coordinates'))
        if has_z:
            polygon_z_counts[gtype]['with_z'] += 1
        else:
            polygon_z_counts[gtype]['without_z'] += 1

print(f"Total features: {len(features)}")
print('\nGeometry types present:')
for gtype, cnt in sorted(type_counts.items(), key=lambda x: (str(x[0]))):
    print(f"- {gtype}: {cnt}")

for ptype in ('Polygon', 'MultiPolygon'):
    if ptype in polygon_z_counts:
        stats = polygon_z_counts[ptype]
        total = stats['with_z'] + stats['without_z']
        print(f"\n{ptype}: {total} features -> {stats['with_z']} with Z, {stats['without_z']} without Z")

if 'Polygon' not in polygon_z_counts and 'MultiPolygon' not in polygon_z_counts:
    print('\nNo Polygon or MultiPolygon features detected in this file.')


Total features: 6801377

Geometry types present:
- Polygon: 6801377

Polygon: 6801377 features -> 0 with Z, 6801377 without Z


^ polygon = building footprint, feature: height

TRIM GEOJSON DB TO BOUNDARY

In [16]:
import geopandas as gpd
from shapely.geometry import Polygon

In [17]:
#load geojson in gpd
try:
    gdf = gpd.read_file(r'\\Nt05\00\0005\CMU_Docs\_Dallas Design Sprint\research\DataSets\BuildingData\w100_n35_w095_n30.geojson')  # Replace with your GeoJSON file path
except Exception as e:
    raise SystemExit(f"Error loading GeoJSON: {e}")

In [18]:
#Define boundary manually for dt Dallas corners
boundary_polygon = Polygon([
    (-96.8084, 32.7873),  # NW corner
    (-96.7940, 32.7873),  # NE corner
    (-96.7940, 32.7740),  # SE corner
    (-96.8084, 32.7740),  # SW corner
    (-96.8084, 32.7873)   # back to NW
])
boundary_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[boundary_polygon])

#share the same CRS
if gdf.crs != boundary_gdf.crs:
    gdf = gdf.to_crs(boundary_gdf.crs)

In [21]:
#filter features within the boundary
filtered_gdf = gdf[gdf.geometry.within(boundary_gdf.unary_union)]

C:\Users\yzheng\AppData\Local\Temp\ipykernel_40072\2887836079.py:2: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.



In [22]:
#results
filtered_gdf.to_file("filtered.geojson", driver="GeoJSON")
print(f"Filtered dataset saved as 'filtered.geojson' with {len(filtered_gdf)} features.")

Filtered dataset saved as 'filtered.geojson' with 247 features.
